In [7]:
import os
import time
from lxml import html
import requests
import numpy as np
import pandas as pd
import time
import re
from tqdm import tqdm
from currency_converter import CurrencyConverter

In [4]:
def scrape(type="CPU", min_year=2017, max_year=2021):
    # Returns a list of tr_elements, one per year
    tr_tables = []

    if type == "CPU":
        base_url = 'https://www.techpowerup.com/cpu-specs/?released='
        filt_url = '&mobile=No&sort=name'
    else:
        base_url = 'https://www.techpowerup.com/gpu-specs/?released='
        filt_url = '&mobile=No&igp=No&sort=name'
    
    years = list(range(min_year, max_year+1))
    for year in tqdm(years):
        page = requests.get(base_url+str(year)+filt_url)
        tree = html.fromstring(page.content)
        tr_elements = tree.xpath('//tr')
        tr_tables.append(tr_elements)

        time.sleep(60)

    return tr_tables

In [64]:
def clean_name(t):
    name = t.text_content()
    aux = name.split()
    
    return " ".join(aux)

def div_positions(tr_tables):
    # Return tuple with Manufacturer and upcoming header position
    div_pos = []

    for tr_elements in tr_tables:
        header_pos = [(clean_name(t), i+1) for i, element in enumerate(tr_elements) for t in element if (clean_name(t) == "AMD") or (clean_name(t) == "Intel") or (clean_name(t) == "NVIDIA")]
        div_pos.append(header_pos)

    return div_pos

In [5]:
cpu_tables = scrape("CPU", 2017, 2021)

100%|██████████| 5/5 [05:03<00:00, 60.76s/it]


In [74]:
def scraped_dataframe(tr_tables):
    div_pos = div_positions(tr_tables)
    row_data = []
    have_header = False

    for year_pos, tr_elements in zip(div_pos, tr_tables):
        # Create header
        if not(have_header):
            _, i = year_pos[0]
            header = ['Manufacturer']
            for t in tr_elements[i]:
                header.append(clean_name(t))
            have_header = True
        
        # Number of manufacturers
        n_man = len(year_pos)
        for i, (manuf, h) in enumerate(year_pos):
            if i == n_man - 1:
                data = tr_elements[h+1:]
            else:
                data = tr_elements[h+1:year_pos[i+1][1]-1]
            manuf_comp = [[clean_name(t) for t in elem] for elem in data]
            for sublist in manuf_comp:
                sublist.insert(0, manuf)
                row_data.append(sublist)
    
    col_data = [[row[i] for row in row_data] for i in range(len(row_data[0]))]
    return pd.DataFrame(dict(zip(header, col_data)))

In [75]:
cpu_df = scraped_dataframe(cpu_tables)
print(cpu_df)

    Manufacturer         Name       Codename    Cores           Clock  \
0            AMD     A10-9700  Bristol Ridge        4  3.5 to 3.8 GHz   
1            AMD    A10-9700E  Bristol Ridge        4    3 to 3.5 GHz   
2            AMD     A12-9800  Bristol Ridge        4  3.8 to 4.2 GHz   
3            AMD    A12-9800E  Bristol Ridge        4  3.1 to 3.8 GHz   
4            AMD      A6-9500  Bristol Ridge        2  3.5 to 3.8 GHz   
..           ...          ...            ...      ...             ...   
402        Intel  Xeon W-3323     Ice Lake-W  12 / 24    3.5 to 4 GHz   
403        Intel  Xeon W-3335     Ice Lake-W  16 / 32    3.4 to 4 GHz   
404        Intel  Xeon W-3345     Ice Lake-W  24 / 48      3 to 4 GHz   
405        Intel  Xeon W-3365     Ice Lake-W  32 / 64    2.7 to 4 GHz   
406        Intel  Xeon W-3375     Ice Lake-W  38 / 76    2.5 to 4 GHz   

          Socket Process L3 Cache    TDP        Released  
0     Socket AM4   28 nm      N/A   65 W  Jul 27th, 2017  
1    

In [76]:
CPU_DATASET = "./Data/cpu_raw.csv"
cpu_df.to_csv(CPU_DATASET, index=False)

In [77]:
gpu_tables = scrape("GPU", 2017, 2022)

100%|██████████| 6/6 [06:04<00:00, 60.67s/it]


In [78]:
print(div_positions(gpu_tables))

[[('AMD', 31), ('NVIDIA', 59)], [('AMD', 31), ('NVIDIA', 49)], [('AMD', 31), ('NVIDIA', 48)], [('AMD', 31), ('Intel', 55), ('NVIDIA', 60)], [('AMD', 31), ('Intel', 45), ('NVIDIA', 49)], [('AMD', 31), ('NVIDIA', 36)]]


In [79]:
gpu_df = scraped_dataframe(gpu_tables)
print(gpu_df)

    Manufacturer               Product Name   GPU Chip        Released  \
0            AMD                 Radeon 550       Lexa  Apr 20th, 2017   
1            AMD       Radeon Instinct MI25    Vega 10  Jun 27th, 2017   
2            AMD     Radeon Pro Duo Polaris  Ellesmere  Apr 24th, 2017   
3            AMD             Radeon Pro SSG    Vega 10   Aug 8th, 2017   
4            AMD            Radeon Pro V320    Vega 10  Jun 29th, 2017   
..           ...                        ...        ...             ...   
217       NVIDIA           GeForce RTX 3050      GA107            2022   
218       NVIDIA  GeForce RTX 3070 Ti 16 GB      GA104        Jan 2022   
219       NVIDIA     GeForce RTX 3080 12 GB      GA102        Jan 2022   
220       NVIDIA  GeForce RTX 3080 Ti 20 GB      GA102        Jan 2022   
221       NVIDIA        GeForce RTX 3090 Ti      GA102         Unknown   

              Bus                  Memory GPU clock Memory clock  \
0     PCIe 3.0 x8     2 GB, GDDR5, 64 bit  

In [80]:
GPU_DATASET = "./Data/gpu_raw.csv"
gpu_df.to_csv(GPU_DATASET, index=False)